In [1]:
# 라이브러리 가져오기
import pandas as pd
import numpy as np

### 계층 색인

- 행, 열의 각 축에 대해 다중 단계(계층)를 지정하여 데이터에 차원을 설정
- 인덱스에 다차원 리스트를 전달하면 계층 색인을 지정할 수 있음
- 데이터 구조를 재배열하거나 pivot 테이블과 같은 그룹 기반 작업에 유용
- 재배열 메서드
    - stack() : 컬럼을 로우로 피벗
    - unstack() : 로우를 컬럼으로 피벗

In [3]:
# Series
# 인덱스에 다차원 리스트(아이템 2개)를 전달
# 다차원 리스트[0] : 상위계층
# 상위계층 작성 시 주의점 : 각 계층별로 속하는 하위계층 값의 개수만큼 계층명 작성
# 상위계층 리스트 개수 = 하위계층 리스트 개수
# 다차원리스트[1] : 하위계층
# 실습)
# 로우 인덱스 - 상위계층 : a, b, c, d
# 하위계층 : a(1, 2, 3), b(1, 2), c(1, 2, 3, 4), d(1)
s1 = pd.Series(np.arange(10), index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'c', 'c', 'd'],
                                    [1, 2, 3, 1, 2, 1, 2, 3, 4, 1]])
s1

a  1    0
   2    1
   3    2
b  1    3
   2    4
c  1    5
   2    6
   3    7
   4    8
d  1    9
dtype: int32

In [4]:
# 인덱스확인
s1.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 2),
            ('c', 1),
            ('c', 2),
            ('c', 3),
            ('c', 4),
            ('d', 1)],
           )

In [5]:
# 상위계층에 접근
# 계층 색인이 적용된 객체에 상위 인덱스에 접근 : 일반적인 Series 인덱싱으로 접근
s1['c']

1    5
2    6
3    7
4    8
dtype: int32

In [6]:
# 계층 색인에 대한 슬라이싱 : 마지막 라벨인덱스도 포함('b':'d'로 입력시 b, c, d 조회)
s1['b':'d']

b  1    3
   2    4
c  1    5
   2    6
   3    7
   4    8
d  1    9
dtype: int32

In [7]:
s1

a  1    0
   2    1
   3    2
b  1    3
   2    4
c  1    5
   2    6
   3    7
   4    8
d  1    9
dtype: int32

In [9]:
# 하위계층에 접근
# a, b, c, d에서 하위계층 라벨번호가 2인 데이터만 조회
# 데이터[상위계층범위, 하위계층라벨번호]
s1[:, 2]

a    1
b    4
c    6
dtype: int32

In [10]:
# 하위 계층에 슬라이싱 : loc 메서드를 이용하여 슬라이싱
# 상위계층이 c이고 하위계층을 2~4까지 슬라이싱(마지막 번호도 포함 : 사용자가 직접 지정한 라벨은 마지막도 포함함)
s1['c'].loc[2:4]

2    6
3    7
4    8
dtype: int32

In [12]:
# unstack() 메서드 : 최하위(기본동작)에 있는 로우 계층을 컬럼으로 적용하여 위로 올림
# Series 객체를 DataFrame 객체로 재배열할 수 있음
# NaN : 기존에는 없었던 로우 계층에 대한 값
s1.unstack()

,1,2,3,4
a,0.0,1.0,2.0,NaN
b,3.0,4.0,NaN,NaN
c,5.0,6.0,7.0,8.0
d,9.0,NaN,NaN,NaN


In [13]:
# stack() 메서드 : 칼럼에 있던 값을 가장 하위 인덱스로 내려서 재배열
# DataFrame을 다시 Series화 할 때 사용
s1.unstack().stack()

a  1    0.0
   2    1.0
   3    2.0
b  1    3.0
   2    4.0
c  1    5.0
   2    6.0
   3    7.0
   4    8.0
d  1    9.0
dtype: float64

In [22]:
# DataFrame 생성
# 구조 : 4 x 5
# 로우 인덱스 : 상위(2017, 2018) / 하위(모든 상위 인덱스에 대해 동일하게 'a', 'b')
# 컬럼 인덱스 : 상위(서울, 경기) / 하위(서울-강남, 잠실 / 경기-분당, 수원, 판교)
# 값 : 1씩 증가하는 20개
df = pd.DataFrame(np.arange(20).reshape(4, 5), index=[[2017, 2017, 2018, 2018], ['a', 'b', 'a', 'b']], 
                  columns=[['서울', '서울', '경기', '경기', '경기'], ['강남', '잠실', '분당', '수원', '판교']]) # columns
df

서울      경기        
        강남  잠실  분당  수원  판교
2017 a   0   1   2   3   4
     b   5   6   7   8   9
2018 a  10  11  12  13  14
     b  15  16  17  18  19

In [23]:
# 컬럼의 상위 계층 접근 : 일반적인 df칼럼 인덱싱 방식
# 서울데이터 조회
df['서울']

강남  잠실
2017 a   0   1
     b   5   6
2018 a  10  11
     b  15  16

In [24]:
# 컬럼이 이중이기 때문에 인덱싱을 이중으로 해야 하나의 요소 조회 가능
df['서울']['강남']

2017  a     0
      b     5
2018  a    10
      b    15
Name: 강남, dtype: int32

In [25]:
# 혹은 튜플로 전달시 역시 자료 조회 가능
df[('서울', '강남')]

2017  a     0
      b     5
2018  a    10
      b    15
Name: (서울, 강남), dtype: int32

In [26]:
# 로우의 상위계층 접근
# 2017년 데이터 조회
df.loc[2017]

서울    경기      
  강남 잠실 분당 수원 판교
a  0  1  2  3  4
b  5  6  7  8  9

In [28]:
# 방법1. 2017 전반기 데이터
df.loc[2017].loc['a']

서울  강남    0
    잠실    1
경기  분당    2
    수원    3
    판교    4
Name: a, dtype: int32

In [29]:
# 방법2. 튜플로 전달(인덱스(row) 조회시 리스트 전달도 허용)
df.loc[(2017, 'a')] # df.loc[[2017, 'a']]도 가능(로우만 됨)

서울  강남    0
    잠실    1
경기  분당    2
    수원    3
    판교    4
Name: (2017, a), dtype: int32

In [30]:
# 분당 - 수원까지 데이터 조회
# 주의점 : 데이터프레임 컬럼은 로우 슬라이싱 결과에 대해서만 슬라이싱 적용 가능
# df['경기']['분당':'수원'] # Erroe -> 컬럼슬라이싱을 단독으로 조회 불가능
df['경기'].loc[:, '분당':'수원']

분당  수원
2017 a   2   3
     b   7   8
2018 a  12  13
     b  17  18

In [31]:
# 로우의 최하위계층(a, b)을 위로 올림 : unstack()
df.unstack()

서울              경기                    
      강남      잠실      분당      수원      판교    
       a   b   a   b   a   b   a   b   a   b
2017   0   5   1   6   2   7   3   8   4   9
2018  10  15  11  16  12  17  13  18  14  19

In [32]:
# 컬럼 최하위를 로우의 하위계층으로 재배열 : stack()
df.stack()

경기    서울
2017 a 강남   NaN   0.0
       분당   2.0   NaN
       수원   3.0   NaN
       잠실   NaN   1.0
       판교   4.0   NaN
     b 강남   NaN   5.0
       분당   7.0   NaN
       수원   8.0   NaN
       잠실   NaN   6.0
       판교   9.0   NaN
2018 a 강남   NaN  10.0
       분당  12.0   NaN
       수원  13.0   NaN
       잠실   NaN  11.0
       판교  14.0   NaN
     b 강남   NaN  15.0
       분당  17.0   NaN
       수원  18.0   NaN
       잠실   NaN  16.0
       판교  19.0   NaN

In [33]:
# 계층의 인덱스번호 또는 라벨을 사용하여 상 하위간 교환
# swaplevel(key1, key2, axis=0(기본값))
# axis가 0인 경우를 row간의상하위 개념을 바꿔준다.
df.swaplevel(1, 0)

서울      경기        
        강남  잠실  분당  수원  판교
a 2017   0   1   2   3   4
b 2017   5   6   7   8   9
a 2018  10  11  12  13  14
b 2018  15  16  17  18  19

In [34]:
# axis=1은 로우 계층 변동이 이난 컬럼 계층 변동
df.swaplevel(1, 0, axis=1)

강남  잠실  분당  수원  판교
        서울  서울  경기  경기  경기
2017 a   0   1   2   3   4
     b   5   6   7   8   9
2018 a  10  11  12  13  14
     b  15  16  17  18  19

### 객체 복사

- 할당기호(=) / 슬라이싱 : 원본과 상호 종속적인 복사본 객체 생성(얕은복사)
- obj.copy() : 원본과 독립적인 복사본 객체 생성(깊은복사)

In [36]:
# 모든 값이 1인 5 x 1 구조의 Series 생성
s1 = pd.Series(np.ones(5))
s1

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
dtype: float64

In [37]:
# 할당기호 = 로 복사 : 얕은 복사
s2 = s1

In [38]:
s2[0] = 10
s2

0    10.0
1     1.0
2     1.0
3     1.0
4     1.0
dtype: float64

In [39]:
# 원본객체의 변동 여부 확인
s1

0    10.0
1     1.0
2     1.0
3     1.0
4     1.0
dtype: float64

In [40]:
# 원본객체의 아이템 수정
s1[1] = 3
s1

0    10.0
1     3.0
2     1.0
3     1.0
4     1.0
dtype: float64

In [42]:
# 복사받은 s2도 영향 받음
s2

0    10.0
1     3.0
2     1.0
3     1.0
4     1.0
dtype: float64

In [43]:
# 슬라이싱으로 복사 : (일반 리스트 -> 깊은 복사, pandas -> 얕은 복사로 취급!)
s3 = s1[:]
s3

0    10.0
1     3.0
2     1.0
3     1.0
4     1.0
dtype: float64

In [44]:
# 복사본 객체 아이템 수정
s3[2] = -9
s3

0    10.0
1     3.0
2    -9.0
3     1.0
4     1.0
dtype: float64

In [45]:
# s1와 s2 확인
print(s1)
print(s2)

0    10.0
1     3.0
2    -9.0
3     1.0
4     1.0
dtype: float64
0    10.0
1     3.0
2    -9.0
3     1.0
4     1.0
dtype: float64


In [46]:
# 원본 변경
s1[3] = 33
s1

0    10.0
1     3.0
2    -9.0
3    33.0
4     1.0
dtype: float64

In [47]:
# 복사본 객체 확인
print(s2)
print(s3)

0    10.0
1     3.0
2    -9.0
3    33.0
4     1.0
dtype: float64
0    10.0
1     3.0
2    -9.0
3    33.0
4     1.0
dtype: float64


In [48]:
# copy()를 이용한 복사(깊은 복사)
s4 = s1.copy()

In [49]:
# s4 수정(s1 영향 x)
s4[4] = 44
s4

0    10.0
1     3.0
2    -9.0
3    33.0
4    44.0
dtype: float64

In [50]:
# 원본 확인 
s1

0    10.0
1     3.0
2    -9.0
3    33.0
4     1.0
dtype: float64

In [51]:
# s1 변경시 역시 s4는 영향 x
s1[0] = 1
s1

0     1.0
1     3.0
2    -9.0
3    33.0
4     1.0
dtype: float64

In [52]:
s4

0    10.0
1     3.0
2    -9.0
3    33.0
4    44.0
dtype: float64

### 정렬

- obj.sort_index() : 인덱스를 기준으로 정렬 (기본값은 ascending=True, 오름차순 정렬)
    - DataFrame 
        - axis = 0 : 기본값, 로우 인덱스 기준으로 정렬
        - axis = 1 : 컬럼 인덱스 기준으로 정렬
- obj.sort_values() : 값을 기준으로 정렬
    - DataFrame 
        - by : 정렬의 기준이 되는 인덱스 값 전달
        - axis = 0 : 기본값, 컬럼을 기준으로 로우 인덱스를 정렬하며 기준값으로 by에 인덱스 컬럼 레벨 또는 컬럼명 전달
        - axis = 1 : 로우 인덱스를 기준으로 컬럼 라벨을 정렬하며 기준값으로 by에 레벨 또는 라벨명 전달

In [53]:
# Series 생성
# 값과 인덱스라벨이 순서대로 들어가지 않은 Series
s1 = pd.Series([2, 3, 1, 7, 0], index=list('gacfd'))
s1

g    2
a    3
c    1
f    7
d    0
dtype: int64

In [55]:
# 인덱스 기준 오름차순으로 정렬
# 기본 동작 : 오름차순 & 로우 인덱스
s1.sort_index()

a    3
c    1
d    0
f    7
g    2
dtype: int64

In [56]:
# 인덱스 기준 내림차순으로 정렬
s1.sort_index(ascending=False)

g    2
f    7
d    0
c    1
a    3
dtype: int64

In [57]:
# 일시적 변경이므로 s1 자체에는 영향없음
s1

g    2
a    3
c    1
f    7
d    0
dtype: int64

In [60]:
# 값 기준 내림차순으로 정렬
s1.sort_values(ascending=False)

f    7
a    3
g    2
c    1
d    0
dtype: int64

In [61]:
# 원본에는 영향을 안 미침
s1

g    2
a    3
c    1
f    7
d    0
dtype: int64

In [62]:
# DataFrame 생성
# 4 x 5, 무작위 정수
# 로우/컬럼 인덱스도 순서가 없는 값 지정
df1 = pd.DataFrame(np.random.randint(20, size=(4, 5)), index=list('hcae'), columns=list('EAFCD'))
df1

,E,A,F,C,D
h,6,3,5,4,18
c,16,18,3,3,9
a,15,10,5,7,7
e,14,4,11,0,16


In [63]:
# 로우 인덱스 기준으로 오름차순 정렬(axis=0)
df1.sort_index()

,E,A,F,C,D
a,15,10,5,7,7
c,16,18,3,3,9
e,14,4,11,0,16
h,6,3,5,4,18


In [64]:
# 로우인덱스 기준으로 내림차순 정렬(axis=0)
df1.sort_index(ascending=False)

,E,A,F,C,D
h,6,3,5,4,18
e,14,4,11,0,16
c,16,18,3,3,9
a,15,10,5,7,7


In [65]:
# 컬럼명 기준으로 오름차순 정렬
df1.sort_index(axis=1)

,A,C,D,E,F
h,3,4,18,6,5
c,18,3,9,16,3
a,10,7,7,15,5
e,4,0,16,14,11


In [66]:
# 컬럼명 기분으로 내림차순 정렬
df1.sort_index(axis=1, ascending=False)

,F,E,D,C,A
h,5,6,18,4,3
c,3,16,9,3,18
a,5,15,7,7,10
e,11,14,16,0,4


In [67]:
# 컬럼 기준으로 내림차순 정렬 후, 로우 기준으로 오름차순 정렬
df1.sort_index(axis=1, ascending=False).sort_index()

,F,E,D,C,A
a,5,15,7,7,10
c,3,16,9,3,18
e,11,14,16,0,4
h,5,6,18,4,3


In [68]:
# 원본에는 영향 없음
df1

,E,A,F,C,D
h,6,3,5,4,18
c,16,18,3,3,9
a,15,10,5,7,7
e,14,4,11,0,16


In [70]:
# 값 기준으로 정렬
# 컬럼 D의 값을 오름차순로 정렬 : 동점자가 나오명면 로우명을 기준으로 정렬
# sort_values(axis=0, by='기준컬럼명') : 기본동작(정렬결과를 행에 반영)
df1.sort_values(by='D')

,E,A,F,C,D
a,15,10,5,7,7
c,16,18,3,3,9
e,14,4,11,0,16
h,6,3,5,4,18


In [73]:
# 컬럼 A를 기준으로 내림차순 정렬
df1.sort_values(by='A', ascending=False)

,E,A,F,C,D
c,16,18,3,3,9
a,15,10,5,7,7
e,14,4,11,0,16
h,6,3,5,4,18


In [76]:
# 인덱스라벨 c의 값을 오름차순으로 정렬
# 결과적으로 정렬되는 대상 : 컬럼
# 정렬의 기준 : 로우 레이블
df1.sort_values(by='c', axis=1)

,F,C,D,E,A
h,5,4,18,6,3
c,3,3,9,16,18
a,5,7,7,15,10
e,11,0,16,14,4


In [78]:
# 로우 'e의 값을 내림차순으로 정렬
df1.sort_values(by='e', axis=1, ascending=False)

,D,E,F,A,C
h,18,6,5,3,4
c,9,16,3,18,3
a,7,15,5,10,7
e,16,14,11,4,0


In [80]:
# 두 개의 컬럼에 대해서 정렬 : 리스트로 묶어서 by의 인자값으로 전달
# 정렬 우선순위 : 차례대로 1순위 > 2순위(1순위 동점시 2순위로 넘어감)
# 1순위 정렬 후 1순위 컬럼의 동점값에 대해 2순위 컬럼의 값이 재정렬
# 동일한 값이 아니라면 무조건 1순위 컬럼이 우선
df1.sort_values(by=['D', 'C'])

,E,A,F,C,D
a,15,10,5,7,7
c,16,18,3,3,9
e,14,4,11,0,16
h,6,3,5,4,18


In [82]:
# E는 오름차순, A는 내림차순으로 하고 싶다면
# ascending[True, False]와 같이 각각의 컬럼에 대해서 ascending을 따로 매겨준다,
df1.sort_values(by=['h', 'c'], axis=1, ascending=[True, False])

,A,C,F,E,D
h,3,4,5,6,18
c,18,3,3,16,9
a,10,7,5,15,7
e,4,0,11,14,16


### 연습문제

#### 1. 아래의 데이터프레임을 생성하세요

- 학생들의 점수는 50 이상 100 미만의 무작위 정수 값을 생성하여 사용

<img src="img/df_sort_practice1.png" width=250 align='left'>

In [94]:
df = pd.DataFrame(np.random.randint(50, 100, size=(5, 4)), index=['Kim', 'Park', 'Lee', 'Jung', 'Moon'], 
                  columns=[[2016, 2016, 2017, 2017], ['영어', '수학', '영어', '수학']])
df

2016     2017    
       영어  수학   영어  수학
Kim    57  54   55  67
Park   84  97   67  82
Lee    94  51   90  92
Jung   58  94   84  68
Moon   86  70   57  85

In [99]:
# 로우 인덱스 설정
#df.index.name = '학생명' # 이건 됨
df.index.set_names('학생명', inplace=True)

In [100]:
df

2016     2017    
       영어  수학   영어  수학
학생명                   
Kim    57  54   55  67
Park   84  97   67  82
Lee    94  51   90  92
Jung   58  94   84  68
Moon   86  70   57  85

In [105]:
# 컬럼 인덱스 이름 설정(리스트로 넘김)
#df.colums.name = ['연도', '과목명'] # 두 개라서 안 붙음
df.columns.set_names(['연도', '과목명'], inplace=True) # inplace=True 반드시 줘야함

In [106]:
df

연도   2016     2017    
과목명    영어  수학   영어  수학
학생명                   
Kim    57  54   55  67
Park   84  97   67  82
Lee    94  51   90  92
Jung   58  94   84  68
Moon   86  70   57  85

#### 2. 2016년 데이터만 별도의 데이터프레임으로 분리 저장하세요.(깊은복사) v

In [111]:
df2016 = df[2016].copy()
df2016

과목명,영어,수학
학생명,,
Kim,57,54
Park,84,97
Lee,94,51
Jung,58,94
Moon,86,70


#### 3. 2016년 데이터에 대해 학생이름을 기준으로 오름차순 정렬하세요.

In [115]:
df2016.sort_index()

과목명,영어,수학
학생명,,
Jung,58,94
Kim,57,54
Lee,94,51
Moon,86,70
Park,84,97


#### 4. 2016년 데이터의 과목별 점수에 대하여 영어 점수가 높은 순서로 정렬하고, 영어 점수 동점자의 경우 수학 점수가 낮은 순서로 정렬하세요

In [122]:
# 2등과 3등의 점수를 동일한 86점으로 변경
df2016['영어']['Park'] = 86
df2016

과목명,영어,수학
학생명,,
Kim,57,54
Park,86,97
Lee,94,51
Jung,58,94
Moon,86,70


In [123]:
df2016.sort_values(by=['영어', '수학'], ascending=[False, True])

과목명,영어,수학
학생명,,
Lee,94,51
Moon,86,70
Park,86,97
Jung,58,94
Kim,57,54


### 데이터 합치기

- merge
    - pandas 객체의 메서드로 pandas.merge(df1, df2 ...)로 사용
    - 두 개의 데이터프레임에 대해 특정 컬럼을 기준으로 합치기
    - 주요 파라미터
        - how : 합치는 방식으로 inner(기본값), left, right, outer 방식 존재
        - on : 합치는 기준으로 두 개의 데이터프레임에 공통으로 존재하는 컬럼명을 사용해야함 (기본값=None)

In [125]:
# 고객번호와 고객명을 저장하고 있는 데이터 프레임 생성
# 딕셔너리 타입으로 데이터프레임 데이터 생성 : key -> col, value -> list() -> row
df1 = pd.DataFrame({'no':[30, 31, 32, 33, 34],
                   'name':['김파이썬', '이장고', '박팬더스', '강넘파이', '최주피터']})
df1

,no,name
0,30,김파이썬
1,31,이장고
2,32,박팬더스
3,33,강넘파이
4,34,최주피터


In [126]:
# 고객번호와 주문수량을 저장하고 있는 데이터 프레임 생성
# 다른 고객번호(no)를 일부 지정
df2 = pd.DataFrame({'no':[30, 33, 40, 41], 'amount':[100, 130, 40, 80]})
df2

,no,amount
0,30,100
1,33,130
2,40,40
3,41,80


In [127]:
# 가장 기본적인 merge 방식 : 공통 컬럼을 기준으로 두 df에 모두 존재하는 교집합만 추출
# 기본 동작 : how = 'inner'
# df1 & df2의 공통칼럼 : no
# 공통 컬럼 내부의 공통 값 : 30, 33
# df1 : 30 ~ 34
# df2 : 30, 33, 40, 41
pd.merge(df1, df2)

,no,name,amount
0,30,김파이썬,100
1,33,강넘파이,130


In [128]:
# outer : 결합 기준으로 공통 컬럼을 사용함
# 교집합이 아닌 부분에 대해서는 NaN으로 처리(누락 x)
# 합집합 형태로 만들어줌
pd.merge(df1, df2, how="outer")

,no,name,amount
0,30,김파이썬,100.0
1,31,이장고,NaN
2,32,박팬더스,NaN
3,33,강넘파이,130.0
4,34,최주피터,NaN
5,40,NaN,40.0
6,41,NaN,80.0


In [129]:
# left : 첫 번째로 전달한 df1의 데이터는 모두 살리고, df2에서는 df1과 겹칠 때만 남김
pd.merge(df1, df2, how="left")

,no,name,amount
0,30,김파이썬,100.0
1,31,이장고,NaN
2,32,박팬더스,NaN
3,33,강넘파이,130.0
4,34,최주피터,NaN


In [130]:
# right : 두 번째 데이터프레임의 데이터를 보존
# df1에서는 df2와 겹치는 경우에 한해 데이터를 남김
pd.merge(df1, df2, how='right')

,no,name,amount
0,30,김파이썬,100
1,33,강넘파이,130
2,40,NaN,40
3,41,NaN,80


In [131]:
# 공통 칼럼이 두 개 이상인 경우
# 고객명, 날짜, 정보를 저장하고 있는 데이터프레임 생성
df3 = pd.DataFrame({'고객명':['김파이썬', '이장고', '박팬더스'], 
                   '날짜':['2020-02-02', '2020-02-15', '2020-02-14'], 
                   '정보':['010', '011', '019']})
df3

,고객명,날짜,정보
0,김파이썬,2020-02-02,010
1,이장고,2020-02-15,011
2,박팬더스,2020-02-14,019


In [132]:
# 고객명, 정보를 저장하고 있는 데이터프레임 생성
df4 = pd.DataFrame({'고객명':['김파이썬', '박팬더스', '최넘파이'], 
                   '정보':['F', 'M', 'M']})
df4

,고객명,정보
0,김파이썬,F
1,박팬더스,M
2,최넘파이,M


In [133]:
# 공통된 이름을 가진 컬럼이 두 개인 경우 파라키터 없이 merge를 실행하면...
pd.merge(df3, df4)

,고객명,날짜,정보


In [134]:
# on 파라미터 : 공통된 컬럼이 여럿인 경우 결합 기준을 지정
# 합칠 때 key로 사용할 컬럼 : '고객명'
# 공통 컬럼 결과 : 고객명, 정보 -> 정보 컬럼 결과를 확인
# 기본동작으로 merge : inner
pd.merge(df3, df4, on='고객명')

,고객명,날짜,정보_x,정보_y
0,김파이썬,2020-02-02,010,F
1,박팬더스,2020-02-14,019,M


In [135]:
# left_on, right_on : 두 개의 데이터프레임에 대해서 서로 다른 기준컬럼을 지정
# 예) 동일힌 속성의 자료를 저장하는 컬럼인데 표기하는 이름이 다른경우
# 고객 이름, 날짜, 구매금액을 저장하고 있는 데이터프레임 생성
df5 = pd.DataFrame({'고객이름':['김파이썬', '박팬더스', '강주피터'], 
                   '날짜':['2020-01-01', '2020-02-01', '2020-02-15'],
                    '구매금액':[1, 2, 3]})
df5

,고객이름,날짜,구매금액
0,김파이썬,2020-01-01,1
1,박팬더스,2020-02-01,2
2,강주피터,2020-02-15,3


In [137]:
# 고객명, 성별을 저장하고 있는 데이터 프레임 생성
df6 = pd.DataFrame({'고객명':['김파이썬', '박팬더스'], 
                   '성별':['F', 'M']})
df6

,고객명,성별
0,김파이썬,F
1,박팬더스,M


In [138]:
# 겹치는 컬럼명이 없어서 에러 발생
pd.merge(df5, df6)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [139]:
# left(df5), right(df6)에서 공통컬럼명을 각각 지정
tmp = pd.merge(df5, df6, left_on='고객이름', right_on='고객명')
tmp

,고객이름,날짜,구매금액,고객명,성별
0,김파이썬,2020-01-01,1,김파이썬,F
1,박팬더스,2020-02-01,2,박팬더스,M


In [146]:
# drop 메서드를 이용해 컬럼 삭제(고객명)
#tmp.drop(columns='고객명') # 나
tmp.drop('고객명', axis=1, inplace=True)

In [147]:
tmp

,고객이름,날짜,구매금액,성별
0,김파이썬,2020-01-01,1,F
1,박팬더스,2020-02-01,2,M


#### concat

- 특정 key를 기준으로 데이터를 합치는 것이 아니라 행, 열 기준으로 데이터를 연결
- 주요 파라미터
    - axis : 0 / 행 방향(기본값)이며 컬럼을 key로 합치고, 1 / 열 방향으로 로우를 key로 합침
    - join : 데이터프레임끼리 연결할 때 합치는 방법으로 outer(기본값), inner 방식 존재
    - ignore_index : 합친 후 기존 인덱스를 유지 또는 새로운 인덱스를 지정

In [149]:
# 공통 인덱스 라벨을 가지는 Series 2개 생성
s1 = pd.Series([1, 2, 3], index=list('abc'))
s2 = pd.Series([5, 6, 7, 8], index=list('abfh'))
print(s1)
print(s2)

a    1
b    2
c    3
dtype: int64
a    5
b    6
f    7
h    8
dtype: int64


In [150]:
# 두 Series간 연결
# 기본 : axis = 0 ( 행방향으로 연결)
# 첫 번째로 전달된 객체가 위에, 두 번째로 전달된 객체가 아래로 추가(연결)
# 인덱스 라벨은 기존 값 유지
pd.concat([s1, s2])

a    1
b    2
c    3
a    5
b    6
f    7
h    8
dtype: int64

In [151]:
# 새로운 인덱스로 초기화하기
pd.concat([s1, s2], ignore_index=True)

0    1
1    2
2    3
3    5
4    6
5    7
6    8
dtype: int64

In [152]:
print(s1, s2)

a    1
b    2
c    3
dtype: int64 a    5
b    6
f    7
h    8
dtype: int64


In [153]:
# 열 방향 연결 -> 두 개의 Series를 연결해서 DataFrame르오 만들기 
# 길이가 다른 경우
pd.concat([s1, s2], axis=1)

E:\anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,0,1
a,1.0,5.0
b,2.0,6.0
c,3.0,NaN
f,NaN,7.0
h,NaN,8.0


In [154]:
# 열 이름을 설정하면서 붙여주기 : keys 파라미터에 컬럼명을 리스트로 전달
pd.concat([s1, s2], axis=1, keys=['c1', 'c2'], sort=False) # sort=False : 경고문을 안 볼 수 있게 해줌

,c1,c2
a,1.0,5.0
b,2.0,6.0
c,3.0,NaN
f,NaN,7.0
h,NaN,8.0


In [158]:
# 두 개의 데이터 프레임 연결
# 고객명, 날짜 구매 금액
df1 = pd.DataFrame({'고객명':['김파이썬', '이장고', '박팬더스'], 
                   '날짜':['2020-01-01', '2020-02-01', '2020-02-15'], 
                   '구매금액':[1, 2, 3]})

# 고객명, 성별
df2 = pd.DataFrame({'고객명':['김파이썬', '최넘파이'],
                    '성별':['F', 'M']})

In [159]:
# 행 방향으로 데이터 프레임 연결(열 개수 증가)
pd.concat([df1, df2], sort=True) # merge와 다르게 []에 넣어주어야 함

,고객명,구매금액,날짜,성별
0,김파이썬,1.0,2020-01-01,NaN
1,이장고,2.0,2020-02-01,NaN
2,박팬더스,3.0,2020-02-15,NaN
0,김파이썬,NaN,NaN,F
1,최넘파이,NaN,NaN,M


In [160]:
# 열 방향으로(열 개수 증가) 데이터 프레임 연결
pd.concat([df1, df2], axis=1)

,고객명,날짜,구매금액,고객명,성별
0,김파이썬,2020-01-01,1,김파이썬,F
1,이장고,2020-02-01,2,최넘파이,M
2,박팬더스,2020-02-15,3,NaN,NaN


In [162]:
df3 = pd.DataFrame({'지역':['서울', '부산', '제주'], 
                   '직업':['웹개발자', '회계사', '회사원']})
df3

,지역,직업
0,서울,웹개발자
1,부산,회계사
2,제주,회사원


In [163]:
# 전제조건, df1의 데이터와 df3의 데이터가 순서상 알맞는 것이 확정적인 경우 사용
pd.concat([df1, df3], axis=1)

,고객명,날짜,구매금액,지역,직업
0,김파이썬,2020-01-01,1,서울,웹개발자
1,이장고,2020-02-01,2,부산,회계사
2,박팬더스,2020-02-15,3,제주,회사원


### append
- 행 방향으로 데|이터를 연결
- 주요 파라미터
    - ignore_index : 합친 후 기존 인덱스를 유지 또는 새로운 인덱스를 지정

In [164]:
# 두 개의 시리즈 생성
s1 = pd.Series([1, 2, 3])
s2 = pd.Series([4, 5, 6, 7])

In [165]:
# 행 방향으로 데이터 연결하고 기존 인덱스 유지
s1.append(s2)

0    1
1    2
2    3
0    4
1    5
2    6
3    7
dtype: int64

In [166]:
# 행방향으로 데이터 연결하고 기존 인덱스 누락시키기
s1.append(s2, ignore_index=True)

0    1
1    2
2    3
3    4
4    5
5    6
6    7
dtype: int64

In [167]:
# 두 개의 데이터프레임
# 서로 다른 컬럼명
df1 = pd.DataFrame([[1, 2], [3, 4]], columns=list('ab'))
df2 = pd.DataFrame([[5, 6], [7, 8], [9, 10]], columns=list('AB'))
print(df1)
print(df2)

   a  b
0  1  2
1  3  4
   A   B
0  5   6
1  7   8
2  9  10


In [168]:
df1.append(df2, sort=False)

,a,b,A,B
0,1.0,2.0,NaN,NaN
1,3.0,4.0,NaN,NaN
0,NaN,NaN,5.0,6.0
1,NaN,NaN,7.0,8.0
2,NaN,NaN,9.0,10.0


In [169]:
# 두 개의 데이터프레임
# 동일한 결과물
df3 = pd.DataFrame([[10, 20], [30, 40], [50, 60]], columns=list('ab'))
df3

,a,b
0,10,20
1,30,40
2,50,60


In [170]:
# 동일한 컬럼명에 대해서 append를 하면 컬럼 개수는 원본 데이터 숫자 그대로, 행만 추가
# append는 보통 이런 케이스에 많이 사용
df1.append(df3)

,a,b
0,1,2
1,3,4
0,10,20
1,30,40
2,50,60


In [171]:
# ignore_index를 걸면 인덱스번호도 0부터 다시 초기화
df1.append(df3, ignore_index=True)

,a,b
0,1,2
1,3,4
2,10,20
3,30,40
4,50,60


In [172]:
# 시리즈와 데이터 프레임
# 데이터프레임에 한 줄 추가할 때 보통 시리즈를 사용함
# 시리즈 라벨 & 데이터프레임 컬럼이 서로 매핑됨
# 시리즈 라벨, 데이터 프레임 컬럼 모두 a, b, c, d로 매칭
s1 = pd.Series([10, 20, 30, 40], index=list('abcd'))
df1 = pd.DataFrame([[1, 1, 1, 1], [2, 2, 2, 2]], columns=list('abcd'))

In [173]:
s1

a    10
b    20
c    30
d    40
dtype: int64

In [174]:
df1

,a,b,c,d
0,1,1,1,1
1,2,2,2,2


In [175]:
# Series의 라벨이 df의 컬럼과 매핑이 됨
# 이 때 series는 df의 새로운 row로 들어감
df1.append(s1, ignore_index=True)

,a,b,c,d
0,1,1,1,1
1,2,2,2,2
2,10,20,30,40


### 집계

- groupby(컬럼명)
    - 특정 속성을 기준으로 묶어서 다양한 집계 함수 적용
    - 대표적인 집계 함수
        - sum : 총합
        - mean : 평균값
        - min : 최소값
        - max : 최대값
        - count : 개수
        - std : 표준편차
- pivot table
    - df.pivot(로우로 사용될 컬럼명, 컬럼으로 사용될 컬럼명, 튜플을 구성하는 값으로 사용될 컬럼명, 집계함수)
    - 일차원으로 컬럼 및 로우가 단순 나열된 형식은 데이터를 파악하는데 적합하지 않기 때문에 pivot을 통해 계층 색인 및 형태 변경을 수행

In [176]:
# 엑셀 데이터 적재
# Os error -> engine='python'
# Unicode, Encoding -> encoding='utf-8 or 'cp949'
data = pd.read_excel('data/인구수예제.xlsx')
data

,도시,자치구,연도,남자인구,여자인구,총인구
0,서울,강남구,2013,73,92,165
1,서울,강남구,2014,139,55,194
2,서울,강남구,2015,123,83,206
3,서울,강남구,2016,147,150,297
4,서울,강남구,2017,57,133,190
5,서울,서대문구,2013,95,111,206
6,서울,서대문구,2014,149,150,299
7,서울,서대문구,2015,106,77,183
8,서울,서대문구,2016,56,109,165
9,서울,서대문구,2017,82,96,178


In [178]:
data.shape

(50, 6)

In [179]:
# 상위자료 5개만 조회
data.head()

,도시,자치구,연도,남자인구,여자인구,총인구
0,서울,강남구,2013,73,92,165
1,서울,강남구,2014,139,55,194
2,서울,강남구,2015,123,83,206
3,서울,강남구,2016,147,150,297
4,서울,강남구,2017,57,133,190


In [180]:
# 하위 자료 5개만 조회
data.tail()

,도시,자치구,연도,남자인구,여자인구,총인구
45,부산,동래구,2013,83,65,148
46,부산,동래구,2014,139,87,226
47,부산,동래구,2015,147,115,262
48,부산,동래구,2016,61,102,163
49,부산,동래구,2017,132,105,237


In [181]:
# 상위 10개 조회
data.head(10)

,도시,자치구,연도,남자인구,여자인구,총인구
0,서울,강남구,2013,73,92,165
1,서울,강남구,2014,139,55,194
2,서울,강남구,2015,123,83,206
3,서울,강남구,2016,147,150,297
4,서울,강남구,2017,57,133,190
5,서울,서대문구,2013,95,111,206
6,서울,서대문구,2014,149,150,299
7,서울,서대문구,2015,106,77,183
8,서울,서대문구,2016,56,109,165
9,서울,서대문구,2017,82,96,178


In [185]:
# 자치구별 남, 여 인구 각각의 총합
# 조사년도는 총 5개년
data.groupby('자치구')[['남자인구', '여자인구']].sum()

,남자인구,여자인구
자치구,,
강남구,539,513
도봉구,485,550
동래구,562,474
동작구,454,582
서대문구,488,543
송파구,415,559
수영구,502,559
영등포구,629,562
종로구,483,373


In [186]:
# 도시별 남여 인구 각각의 통합
data.groupby('도시')[['남자인구', '여자인구']].sum()

,남자인구,여자인구
도시,,
부산,1684,1548
서울,3493,3682


In [187]:
# 연도, 도시별 남여 인구 통합
data.groupby(['연도', '도시'])[['남자인구', '여자인구']].sum()

남자인구  여자인구
연도   도시            
2013 부산   341   262
     서울   758   769
2014 부산   314   369
     서울   882   691
2015 부산   331   266
     서울   541   710
2016 부산   276   376
     서울   671   722
2017 부산   422   275
     서울   641   790

In [189]:
# 도시별 연도별 총인구 평균을 구해주세요
data.groupby(['도시', '연도'])[['총인구']].mean() # 이중리스트 : dataframe, 리스트 : 그냥 data

총인구
도시 연도              
부산 2013  201.000000
   2014  227.666667
   2015  199.000000
   2016  217.333333
   2017  232.333333
서울 2013  218.142857
   2014  224.714286
   2015  178.714286
   2016  199.000000
   2017  204.428571

In [191]:
# 데이터 파악에 용이한 구조로 변경
data.groupby(['도시', '연도'])[['총인구']].mean().unstack(0) # 도시 : 0번 인덱스, 연도 : 1번 인덱스

총인구            
도시            부산          서울
연도                          
2013  201.000000  218.142857
2014  227.666667  224.714286
2015  199.000000  178.714286
2016  217.333333  199.000000
2017  232.333333  204.428571

#### groupby 결과를 pivot_table로 실행할 수 있다.

In [192]:
# 연도별 & 도시별 & 남여 인구 각각의 최대값
# pivot_table(index=로우로 사용할 컬럼명, columns=컬럼으로 사용할 컬럼명,
# values=집계함수를 적용할 값으로 사용할 컬럼, aggfunc='사용할 집계함수 이름')
# 로우 : 연도, 도시
# 사용할 값 : 남자인구, 여자인구
# 집계함수 : max
# groupby => data.groupby(['연도', '도시'])[['남자인구', '여자인구']]max()
data.pivot_table(index=['연도', '도시'], values=['남자인구', '여자인구'], aggfunc='max')

남자인구  여자인구
연도   도시            
2013 부산   134   103
     서울   146   138
2014 부산   139   144
     서울   149   150
2015 부산   147   115
     서울   123   139
2016 부산   134   148
     서울   147   150
2017 부산   146   105
     서울   145   150

In [193]:
# .unique()는 출현한 요소를 한 번씩만 출력한다.
data.도시.unique()

array(['서울', '부산'], dtype=object)

In [194]:
# 도시별, 연도별 총인구 평균
# 로우 : 연도
# 컬럼 : 도시
# 값 : 총인구
# 집계 : 평균
data.pivot_table(index=['연도'], columns=['도시'], values=['총인구'], aggfunc='mean')

총인구            
도시            부산          서울
연도                          
2013  201.000000  218.142857
2014  227.666667  224.714286
2015  199.000000  178.714286
2016  217.333333  199.000000
2017  232.333333  204.428571

In [195]:
# 여러 개의 집계함수를 서로 다른 값에 사용
# 도시별, 자치구별 남자인구의 평균, 여자인구의 합계
# 로우 : 도시, 자치구
# 값 : 남자인구, 여자인구
# 집계함수 : 남자: 평균, 여자 : 총합
tmp = data.pivot_table(index=['도시', '자치구'], values=['남자인구', '여자인구'], 
                      aggfunc={'남자인구': 'mean', '여자인구':'sum'})

In [202]:
tmp.columns = ['남자인구_평균', '여자인구_총합']

In [203]:
tmp

남자인구_평균  여자인구_총합
도시 자치구                   
부산 동래구     112.4      474
   수영구     100.4      559
   해운대구    124.0      515
서울 강남구     107.8      513
   도봉구      97.0      550
   동작구      90.8      582
   서대문구     97.6      543
   송파구      83.0      559
   영등포구    125.8      562
   종로구      96.6      373

### pivot_table 실습

In [206]:
# 1. 도시별 자치구별 남자인구 평균(data변수 활용하기)
p1 = data.pivot_table(index=['도시', '자치구'], values=['남자인구'], aggfunc='mean')
p1

남자인구
도시 자치구        
부산 동래구   112.4
   수영구   100.4
   해운대구  124.0
서울 강남구   107.8
   도봉구    97.0
   동작구    90.8
   서대문구   97.6
   송파구    83.0
   영등포구  125.8
   종로구    96.6

In [209]:
# 남자인구 컬럼을 남자인구_평균 컬럼으로 이름 변경하기
p1.columns = ['남자인구_평균']
p1

남자인구_평균
도시 자치구          
부산 동래구     112.4
   수영구     100.4
   해운대구    124.0
서울 강남구     107.8
   도봉구      97.0
   동작구      90.8
   서대문구     97.6
   송파구      83.0
   영등포구    125.8
   종로구      96.6

In [210]:
# 바뀌었는지 상위 3개만 출력해 확인(head 활용)
p1.head(3)

남자인구_평균
도시 자치구          
부산 동래구     112.4
   수영구     100.4
   해운대구    124.0

In [215]:
# 2. 도시별 자치구별 평균 남자인구 구하기
p2 = data.pivot_table(index=['도시', '자치구'], values=['남자인구'], aggfunc='mean')
p2

남자인구
도시 자치구        
부산 동래구   112.4
   수영구   100.4
   해운대구  124.0
서울 강남구   107.8
   도봉구    97.0
   동작구    90.8
   서대문구   97.6
   송파구    83.0
   영등포구  125.8
   종로구    96.6

In [222]:
# 많은 순서대로 5개 출력(head 활용) #v
p2.sort_values(by='남자인구', ascending=False).head()

남자인구
도시 자치구        
서울 영등포구  125.8
부산 해운대구  124.0
   동래구   112.4
서울 강남구   107.8
부산 수영구   100.4

### 통계

- 주요 통계 함수
    - value_count() : 각 고유값의 중복개수
    - count : NaN 값을 제외한 값의 개수
    - describe : 각 컬럼에 대한 요약통계 계산(count, mean, std, min, 1사분위수, 중위값, 3사분위수, max)
    - min, max : 최소, 최대 값
    - sum : 총 합
    - cumsum : 누적합
    - mean : 평균
    - median : 중위값(전체 데이터를 나열 했을 때 중간에 위치한 값)
    - var : 분산(데이터가 전체적으로 흩어진 정도, 편차제곱의 평균)
    - std : 표준편차 = 분산의 양의 제곱근
- 주요 파라미터
    - axis : 연산의 기준이 되는 축, axis=0(기본값)이면 행 방향으로 axis=1이면 열 방향으로 적용
    - skipna : NaN 값을 제외할지 여부를 설정, 기본값 = True

In [229]:
# 샘플 데이터 생성
df = pd.DataFrame(np.random.randint(50, 100, (5, 4)),
                 index=['Kim', 'Park', 'Lee', 'Jung', 'Moon'],
                 columns = [[2016, 2016, 2017, 2017], ['영어', '수학', '영어', '수학']])
df.index.set_names('학생명', inplace=True)
df.columns.set_names(['연도', '과목'], inplace=True)
df

연도   2016     2017    
과목     영어  수학   영어  수학
학생명                   
Kim    78  84   69  56
Park   60  92   92  91
Lee    95  91   97  75
Jung   79  88   91  65
Moon   74  93   86  94

In [230]:
# 요약 통계
df.describe()

연도          2016                  2017          
과목            영어         수학         영어        수학
count   5.000000   5.000000   5.000000   5.00000
mean   77.200000  89.600000  87.000000  76.20000
std    12.517987   3.646917  10.793517  16.36154
min    60.000000  84.000000  69.000000  56.00000
25%    74.000000  88.000000  86.000000  65.00000
50%    78.000000  91.000000  91.000000  75.00000
75%    79.000000  92.000000  92.000000  91.00000
max    95.000000  93.000000  97.000000  94.00000

In [231]:
# 2017년도 성적만 분리해서 df2017로 저장(깊은복사)
df2017 = df[2017].copy()
df2017

과목,영어,수학
학생명,,
Kim,69,56
Park,92,91
Lee,97,75
Jung,91,65
Moon,86,94


In [232]:
df2017.shape

(5, 2)

In [233]:
# 1) count - NaN을 제외한 데이터의 개수(행 개수)
# 축 기준 결정 : axis를 통해 행/열 개수인지 지정가능
# 행 개수 : axis=0
df2017.count()

과목
영어    5
수학    5
dtype: int64

In [234]:
# 열 개수 -> axis=1
df2017.count(axis=1)

학생명
Kim     2
Park    2
Lee     2
Jung    2
Moon    2
dtype: int64

In [235]:
# value_counts() : 범주형 데이터(문자열 등)에 대한 각 분류값의 개수
# 도시, 자치구 데이터 사용(data)
data['도시'].value_counts()

서울    35
부산    15
Name: 도시, dtype: int64

In [236]:
df2017

과목,영어,수학
학생명,,
Kim,69,56
Park,92,91
Lee,97,75
Jung,91,65
Moon,86,94


In [237]:
# 2) sum() : 값의 총합을 계산
# 기본 축 방향 : 열별 총합
# == 과목별 총합
# axis=0
df2017.sum()

과목
영어    435
수학    381
dtype: int64

In [238]:
# 학생별 점수 총합
df2017.sum(axis=1)

학생명
Kim     125
Park    183
Lee     172
Jung    156
Moon    180
dtype: int64

In [239]:
# 3) mean()
# 과목별 평균
# axis=0 (디폴트)
df2017.mean()

과목
영어    87.0
수학    76.2
dtype: float64

In [240]:
# 학생별 과목 평균 점수를 구해주세요
df2017.mean(axis=1)

학생명
Kim     62.5
Park    91.5
Lee     86.0
Jung    78.0
Moon    90.0
dtype: float64

In [241]:
df

연도   2016     2017    
과목     영어  수학   영어  수학
학생명                   
Kim    78  84   69  56
Park   60  92   92  91
Lee    95  91   97  75
Jung   79  88   91  65
Moon   74  93   86  94

In [ ]:
# 문제1 : df변수를 이용해서 2016, 2017 년도별, 과목별 평균성적 출력 v

In [249]:
df.mean()

연도    과목
2016  영어    77.2
      수학    89.6
2017  영어    87.0
      수학    76.2
dtype: float64

In [ ]:
# 위의 년도별, 과목별 평균의 로우가 연도로, 컬럼이 과목이 되도록 수정해주세요

In [250]:
df.mean().unstack()

과목,수학,영어
연도,,
2016,89.6,77.2
2017,76.2,87.0


In [ ]:
# 문제2 : df 변수를 이용해서 학생별로 전체평균을 출력

In [251]:
df.mean(axis=1)

학생명
Kim     71.75
Park    83.75
Lee     89.50
Jung    80.75
Moon    86.75
dtype: float64